# 3. Flow control

1. [introduction](#introduction)
1. [Boolean logic](#boolean logic)
1. [Exceptions](#exception handling)
1. [iterators](#iterators)
1. [Break-continue-pass](#break-cont-pass)
1. Exercise: [prime sift](#primes)

## 3.1 Introduction <a name="introduction" />

### 3.1.1 Motivating example: sample from a distribution

Consider a distribution with the following expression
$$
f_X(x) = \begin{cases}
2 - \frac{1}{2x^2} & \text{if } 1/2<x\leq 1\\
\frac{2}{z^2} - \frac{1}{2} & \text{if } 1<x\leq 2\\
0 & \text{otherwise}
\end{cases}
$$

Sample from the distribution using a Monte Carlo sampler.

In [ ]:
# import a plotting library for showing the results
from matplotlib import pyplot as plt
%matplotlib notebook

# list comprehension: simple generation of a list with a for loop
X = [3/2*x/1000 + 1/2 for x in range(1000)] # range(1000) is a generator
fX = [2 - 1/(2 * x**2) if x < 1 else 2 / x**2 - 1/2 for x in X]
fig, ax = plt.subplots()
ax.plot(X, fX)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f_X$')
ax.set_title(r'probability density function of $x$'); # ; to suppress the output to std out of last line

In [ ]:
n_samples = 100000 # use 1e4 samples
Xbins = [3/2*x/100 + 1/2 for x in range(100)] # range(1000) is a generator
# draw samples uniformly over the square [1/2, 2] x [0, 3/2]
import random
samples = [(random.uniform(1/2, 2), random.uniform(0, 3/2)) for _ in range(n_samples)]
# filter the samples, only samples for which the y-coordinate is under the curve should be retained
samples = [s[0] for s in samples if s[1] < min(2 - 1/(2 * s[0]**2), 2 / s[0]**2 - 1/2)]
fig2, ax2 = plt.subplots()
ax2.hist(samples, bins=Xbins)
ax2.set_xlabel(r'$x$')
ax2.set_ylabel('counts')
ax2.set_title('observed frequencies of sample')
print('percent of samples accepted: {:.1f}%'.format(len(samples)/n_samples*100))
print('theoretical acceptance rate: {:.1f}%'.format(4/9*100))

### 3.1.2 key ingredients for flow control

1. conditional rerouting: indicates with program path is executed, given a condition is (un)satisfied
1. an infinite loop: repeats until conditional rerouting tells otherwise
1. a final condition: indicates when to terminate the iterations

Python has two ways for **conditional rerouting**:
* `if`/`elif`/`else` with condiotional statements 
* throw an `Exception` and handle it properly (`try` &#8230; `except` &#8230; `finally`)

The **infinite loop** is created by 
```python
while True:
    <statements> # obligatory 4-space indentation!
```

A **final condition** can be
* an exception `Exception:StopIteration`
* a `break`, `continue`, or `return` statement

## 3.2 conditional rerouting with boolean logic <a name="boolean logic"/>

### 3.2.1 Getting acquainted with boolean expressions in python

#### basic boolean values and operators

* 2 boolean values : `True` and `False`

* comparison operators : `==`,&ensp; `>`,&ensp; `<`,&ensp; `<=`,&ensp; `>=`,&ensp; `~=`,&ensp; `is`

* negation: `not`,&ensp; `!`

#### comparison operators: _to be or not to be_

| operator | meaning |
|:-------- |:----|
| `==` | __similar__ _or_ equal in __value__|
| `is` | __identical__ _or_ same __object__ |

_For fancier string layout when printing to stdout, see [input output](https://docs.python.org/3.6/tutorial/inputoutput.html)_

#### combining logical statements through operators

* joining statements &rightarrow; logical operations

    * `and`, `or`
    * bitwise: `&` (and), `|` (or), `^` (xor)
    
#### implicit boolean logic

* predicates are used in flow control `if`, `while`

* all following statements evaluate to `True`

  ```python
  0 == False
  0 is not False
  1 == True
  1 is not True
  2 != True
  not 2 == True
  bool(2) == True
  bool(2) is True
  bool([1, 2]) is True
  bool([]) is False
  ```

In [ ]:
print(True and False)
print(True or True and False)  # operator precedence?
print(bin(0b1100 ^ 0b1010))    # binary
print(bin(12 ^ 10))            # explain!

### 3.2.2 trickier behaviour when not dealing with logical expressions

In [ ]:
print(1 and 2)
print(bool(1) and bool(2))
print(True and "print me")  # if the first statement evaluates to True print second
print(False and "print me") # if the first statement evaluates to False print False
print("print me" and True)  # non-commutative!
print("print me" and False) 

### 3.2.3 compound statements

* compound statement = header + suite

* separator `:`

* suite &rightarrow; __4 space__ indentation!

    ```python
    if True:
        print("execute me")
    ```

  * header:<br />
    &bbrk;&bbrk;&bbrk;&bbrk;suite
    
#### `if`/`elif`/`else` statement

In [ ]:
if True:
    print('execute me')
if False:
    print('do not exectute me')
if not False:
    print('double negation is an affirmation')

In [ ]:
x = 1
if x is 1:
    print('x identifies with 1')
elif x == 1: # Auch ! although True this does not get evaluated
    print('the value of x is 1')
else:
    print('x has nothing to do with 1')

#### `while` statement

_HINT_: you can interrupt kernel execution with <kbd>Ctrl</kbd>+<kbd>C</kbd> or interupt the kernel by <kbd>ESC</kbd>+<kbd>I</kbd> or use the menu _Kernel_ &rightarrow; _Interrupt_

In [ ]:
while True: # infinite loop (no final condition), no user feedback
    pass

In [ ]:
x = 1
while True:
    try:
        x /= 2
        if x < 0.01:
            raise StopIteration
    except StopIteration:
        break # breaks here, but executes 'finally' suite first
    finally:
        print("Current value x = {:.3f}".format(x))    

In [ ]:
while True:
    try:
        n = int(input("Give an integer value: "))
        break
    except ValueError:
        print("Auch! Try again ...")

#### Exercise: buffer

* implement a _last in first out_ (LIFO) buffer

In [ ]:
l = list(range(10))   # list with 10 elements from 0 to 9
print(l)              
while l:              # bool of empty list is False, otherwise True
    element = l.pop() # pop last element from list (LIFO or stack)
    print('popped element {} from list'.format(element))
    print('list = {}'.format(l))

* implement a _first in first out_ (FIFO) buffer<br />
  you might want to have a look at [data structures](https://docs.python.org/3.6/tutorial/datastructures.html)

In [ ]:
l = list(range(10))
print(l)
# complete

## 3.3 Exception handling <a name="exception handling" />

### 3.3.1 the `try` &#8230; `except` &#8230; `finally` triplet

* Exception ("error") &rightarrow; if not handled, code execution is stopped

* handling Exceptions &rightarrow; `try` &#8230; `except` (&#8230; `finally`)

In [ ]:
a = 1
b = 0
a / b

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    c = a / b
except:
    print("Exception was raised, but which one ?")

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    c = a / b
except:
    print("Exception was raised, but which one ?")
    
# if the exception is raised, code is halted, no execution of the next line
print("I'm Always present at output, whatever exception raised")

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    c = a / b
except:
    print("Exception was raised, but which one ?")
finally:    
    print("I'm Always present at output, whatever exception raised")

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    a / b
except Exception as e:   # we can label the exception to reuse later
    print("First print me, then raise \"{}\" exception again".format(e))
    raise                      # raises last exception again

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    a / b
except ZeroDivisionError as e: # only division-by-zero errors
    print("I'm printed when a division by zero occurs: \"{}\"".format(e))
    raise                      # raises last exception again
except Exception as e: # all other errors
    print("I'm printed for any other exception: here \"{}\"".format(e))
    raise
finally:
    print("I'm Always present at output, whatever exception raised")

In [ ]:
a = 1
b = 0
try:
    a = 1
    b = 0
    a[0] # try "a[0]" instead
except ZeroDivisionError as e: # only division-by-zero errors
    print("I'm printed when a division by zero occurs: \"{}\"".format(e))
    raise                      # raises last exception again
except Exception as e: # all other errors
    print("I'm printed for any other exception: here \"{}\"".format(e))
    raise
finally:
    print("I'm Always present at output, whatever exception raised")

### 3.3.2 Raise an Exception on your own

You may also want to raise your own Exception, see [errors and exceptions](https://docs.python.org/3.6/tutorial/errors.html)

In [ ]:
a = 1
b = 0
try:
    raise StopIteration # all code after this Exception in the 'try' suite will not get executed
    a = 11
    b = 10
except StopIteration:
    print('You wanted to stop the execution of your code')
finally:
     print('Current values of a and b: a={} and b={}\n'.format(a, b))

a = 1
b = 0
try:
    a = 11
    b = 10
    raise StopIteration
except StopIteration:
    print('You wanted to stop the execution of your code')
finally:
     print('Current values of a and b: a={} and b={}'.format(a, b))

#### Exercise: your self-designed for-loop

Define a for-loop using an infinite `while` and the `StopIteration` exception. The for-loop should print the 26 letters of the alphabet one by one on each line

In [ ]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
# complete

## 3.4 iterators <a name="iterators"/>

* __iterable__: returns its members one after another

    * `__getitem__()`

* __iterator__: datastream returning member after member

    * `__next__()`

* __generator__: function yielding a series of objects

    * `yield`

Note:
generators require less memory, iterable not saved in memory

![iterables, iterators, generators, &#8230;](./images/itergen.png)

<div align='right'>Taken from [nvie.com](http://nvie.com/posts/iterators-vs-generators/)</div>

### 3.4.1 `range`

* `range(start, stop, step)` from start to stop with stepsize `step` (stop excluded!)

    * `range(5)` &sim; `[0, 1, 2, 3, 4]`
    
    * `range(2, 5)` &sim; `[2, 3, 4]`
    
    * `range(0, 5, 2)` &sim; `[0, 2, 4]`


* `iterator`, not an `iterable`

    * `range(5)` is an iterator
    
    * `list(range(5))` transforms iterator into iterable
    

**Note:** `range(5)` cannot be indexed (it is not an iterable!)

In [ ]:
l = list(range(10))        # this is an iterable
it = l.__iter__()          # this is an iterator
print(it.__next__())       # getting elements, one by one 
print(it.__next__())
print(it.__next__())

while it:
    print(it.__next__())   # generates a StopIteration exception when no more elements

In [ ]:
l = list(range(10))        # this is an iterable

it2 = l.__iter__()
try:
    while it2:
        print(it2.__next__())
except StopIteration:
    pass
finally:
    print("no more elements in it2")

#### But wait! ... this time we've truly reinvented the for-loop

In [ ]:
for element in range(10):
    print(element)
print("no more elements in the iterator")

### 3.4.2 list comprehension (very, very Pythonic)

* constructing lists from iterators

In [ ]:
# general for
y = list()
for x in range(10):
    y.append(x ** 2)
print(y)

# list comprehension
z = [x ** 2 for x in range(10)] # z = y
print(z)

* more involved including conditional statements

In [ ]:
w = [(x, y) for x in [1, 2, 3] for y in [1, 2, 3] if x < y]
print(w)
t = [(x, y) if x < y else (x, ) for x in [1, 2, 3] for y in [1, 2, 3]]
print(t)

## 3.5 `break`, `continue`, and `pass` <a name="break-cont-pass"/>

* `break`: breaks out of innermost `for`- or `while`-loop

    * `else` executed if no break occurs


* `continue`: continues with next iteration of loop


* `pass`: does nothing (but syntactical necessity)

In [ ]:
try:
    1/0
except ZeroDivisionError:
    pass # do nothing special, just be aware that we might have encountered a ZeroDivisionError
finally:
    print("all is ok, we continue as if nothing happened")

## Exercise: Prime sift <a name="primes" />
#### Print all prime numbers below 1000
_HINT_: use `for`, `break`, and `else` and a dynamically growing `list`